<a href="https://colab.research.google.com/github/saurabhsingh1411/imdb_dnn_keras/blob/master/Imdb_dataset_DNN_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Lib and datsets

In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_datasets as tfds

In [9]:
dataset=pd.read_csv('IMDB Dataset.csv')

In [10]:
dataset['sentiment'].value_counts().plot

In [11]:
negative = len(dataset[dataset['sentiment']=='positive'])
positive = len(dataset) - negative

print('Positive reviews are {} and negative reviews are {} of total {} '.format(positive,negative,len(dataset)))

Positive reviews are 25000 and negative reviews are 25000 of total 50000 


In [12]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
le=LabelEncoder()

X_train,X_test,y_train,y_test=train_test_split(dataset['review'].values,dataset['sentiment'].values,test_size=0.2)

In [14]:
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [15]:
y_test=np.array(y_test)

In [16]:
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Preprocessing Text

In [17]:
tokenizer=Tokenizer(num_words=10000,oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index

training_seq=tokenizer.texts_to_sequences(X_train)

testing_seq=tokenizer.texts_to_sequences(X_test)

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

training_pad_seq=pad_sequences(training_seq,maxlen=200,truncating='post',padding='pre')
testing_pad_seq=pad_sequences(testing_seq,maxlen=200,truncating='post',padding='pre')

In [19]:
print('total unique words',len(word_index))

total unique words 112254


## Model Buiulding

In [20]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(len(word_index) + 1,200,input_length=200,
                            trainable=False),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(256,activation = 'relu',),
                             tf.keras.layers.Dense(128,activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1,activation = tf.nn.sigmoid)])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 200)          22451000  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               135680    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

In [22]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy() , optimizer='Adam' , metrics = 'accuracy')

In [ ]:
history = model.fit(training_pad_seq,y_train,epochs = 10 ,validation_data=(testing_pad_seq,y_test))

Epoch 1/10
 928/1250 [=====================>........] - ETA: 1:00 - loss: 0.6242 - accuracy: 0.6539

In [ ]:
print(type(training_pad_seq))
print(type(y_train))

In [ ]:
print(type(testing_pad_seq))
print(type(y_test))

plotting accuracies

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)

plt.show()